Задание.

2. Загрузите данные в ClickHouse и создайте таблицу для их хранения.

In [1]:
!pip install docker

In [2]:
!pip install clickhouse_driver

In [15]:
!docker run -d --name clickhouse-server --ulimit nofile=262144:262144 -p 9000:9000 yandex/clickhouse-server

docker: Error response from daemon: Conflict. The container name "/clickhouse-server" is already in use by container "b6bfb2ca92e3990875262c1361821e2059134eb9e17b5466d8790451ea49c0bc". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [16]:
from clickhouse_driver import Client

client = Client(host='localhost',  # Use 'localhost' or '127.0.0.1' for a local server
                user='default',    # Default user, adjust if you've changed the user
                password='',       # Default installation has no password for 'default' user
                port=9000)         # Default TCP port for ClickHouse

# Attempt to execute a query
try:
    result = client.execute('SHOW TABLES')
    print(result)
except Exception as e:
    print(f"Encountered an error: {e}")

[('trips',)]


In [19]:
client.execute('CREATE DATABASE IF NOT EXISTS scraped_books')

[]

In [43]:
client.execute('SHOW TABLES')

[('trips',)]

In [42]:
client.execute('DROP TABLE IF EXISTS scraped_books.bookshelf')

[]

In [44]:
client.execute (
'''
CREATE TABLE IF NOT EXISTS scraped_books.bookshelf
(
    name String,
    price String,
    avaiability UInt8,
    url String,
    quantity UInt8
)
ENGINE = MergeTree
ORDER BY name;
'''
)

print("Таблица создана успешно.")

Таблица создана успешно.


In [45]:
import json

In [46]:
with open('D:/GeekBrains/DATA ENGINEER/Сбор и разметка данных/Conda_projects/l2_s2_t2_hw.json', 'r') as file:
    data = json.load(file)

In [47]:
for dict in data:
    client.execute("""
        INSERT INTO scraped_books.bookshelf (
            name, price, avaiability, url, quantity
        ) VALUES""",
        [(dict["name"] or "",
        dict["price"] or "",
        dict["avaiability"] or "",
        dict["url"] or "",
        dict["quantity"] or "")])

print("Данные введены успешно.")